<a href="https://colab.research.google.com/github/paulvoliva/Public-PDE-Solvers/blob/main/pde_solver_GAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install numpy==1.19.4

     |████████████████████████████████| 14.5MB 309kB/s 
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Successfully uninstalled numpy-1.19.5


In [1]:
pip install torch==1.6.0+cu101 torchvision==0.7.0+cu101 -f https://download.pytorch.org/whl/torch_stable.html


Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 708.0MB 27kB/s 
     |████████████████████████████████| 5.9MB 25.5MB/s 
  Found existing installation: torch 1.7.0+cu101
    Uninstalling torch-1.7.0+cu101:
      Successfully uninstalled torch-1.7.0+cu101
  Found existing installation: torchvision 0.8.1+cu101
    Uninstalling torchvision-0.8.1+cu101:
      Successfully uninstalled torchvision-0.8.1+cu101


In [2]:
!pip install ray==1.0.1.post1

     |████████████████████████████████| 23.1MB 140kB/s 
     |████████████████████████████████| 3.2MB 42.6MB/s 
     |████████████████████████████████| 71kB 9.0MB/s 
     |████████████████████████████████| 81kB 10.2MB/s 
     |████████████████████████████████| 204kB 41.2MB/s 
     |████████████████████████████████| 71kB 8.2MB/s 
     |████████████████████████████████| 133kB 45.8MB/s 
     |████████████████████████████████| 1.3MB 28.9MB/s 
     |████████████████████████████████| 61kB 8.2MB/s 
     |████████████████████████████████| 143kB 40.6MB/s 
     |████████████████████████████████| 296kB 30.9MB/s 
     |████████████████████████████████| 102kB 9.7MB/s 
  Created wheel for gpustat: filename=gpustat-0.6.0-cp36-none-any.whl size=12622 sha256=f09b3fb4f56dbc6253f1d40fa1aecd721798358d1494e14b99839451723b83ea
  Stored in directory: /root/.cache/pip/wheels/48/b4/d5/fb5b7f1d040f2ff20687e3bad6867d63155dbde5a7c10f4293
  Created wheel for idna-ssl: filename=idna_ssl-1.1.0-cp36-none-any.whl size

In [3]:
!pip install tensorboardX

     |████████████████████████████████| 317kB 4.7MB/s 


In [ ]:
!nvidia-smi

Thu Jan 21 10:34:01 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P8    28W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# -*- coding: utf-8 -*-
"""PDE Solver.ipynb
Here we will take the time components in the third axis
"""

import math
import torch
import os
from ray import tune
from ray.tune.schedulers import ASHAScheduler
from torch.autograd import grad
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np

"""# Exact solution u(x) for the example PDE
we conduct a experiment on solving a IBVP with nonlinear diffusion-reaction equation and boundary condition involving time:
\begin{equation}
\left\{\begin{array}{ll}
u_{t}-\Delta u-u^{2}=f(x, y, t), & \text { in } \Omega \times[0, T] \\
u(x, y, t)=g(x, y, t), & \text { on } \partial \Omega \times[0, T] \\
u(x, y, 0)=h(x, y), & \text { in } \Omega
\end{array}\right.
\end{equation}
where $\Omega=(-1,1)^{2} \subset \mathbb{R}^{2}$. In this test, we give the definition of $f(x,y,t)=\left(\pi^{2}-2\right) \sin \left(\frac{\pi}{2} x\right) \cos \left(\frac{\pi}{2} y\right) e^{-t} - 4 \sin ^{2}\left(\frac{\pi}{2} x\right) \cos \left(\frac{\pi}{2} y\right) e^{-2 t}$ in $\Omega \times[0, T]$, $g(x,y,t)=2 \sin \left(\frac{\pi}{2} x\right) \cos \left(\frac{\pi}{2} y\right) e^{-t}$ on $\partial \Omega \times[0, T]$ and $h(x,y)=2 \sin \left(\frac{\pi}{2} x\right) \cos \left(\frac{\pi}{2} y\right)$ in $\Omega$. And the true solution is $u(x,y,t)=2 \sin \left(\frac{\pi}{2} x\right) \cos \left(\frac{\pi}{2} y\right) e^{-t}$.

# PDE Setup
"""

'''
# I use this to get the shape of the tensor when I debug
old_repr = torch.Tensor.__repr__
def tensor_info(tensor):
    return repr(tensor.shape)[6:] + ' ' + repr(tensor.dtype)[6:] + '@' + str(tensor.device) + '\n' + old_repr(tensor)
torch.Tensor.__repr__ = tensor_info
#'''

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(device)
print(torch.cuda.is_available())

# TODO: implement cuda

def func_u_sol(xt):
    l = xt.shape[0]
    u = 2 * torch.sin(math.pi / 2 * xt[:, 0, :]) * torch.cos(math.pi / 2 * xt[:, 1, :]) * torch.exp(xt[:, 2, :])
    return(u)


# We denote spatial coordinates with time as 'xt' and 'x' without

def func_f(xt):
    l = xt.shape[0]
    f = (math.pi ** 2 - 2) * torch.sin(math.pi / 2 * xt[:, 0, :]) * torch.cos(math.pi / 2 * xt[:, 1, :]) * torch.exp(
        -xt[:, 2, :]) - 4 * torch.sin(math.pi / 2 * xt[:, 0, :]) ** 2 * torch.cos(math.pi / 2 * xt[:, 1, :]) * torch.exp(-xt[:, 2, :])
    return(f)


def func_g(boundary_xt):
    return func_u_sol(boundary_xt)


def func_h(x):
    h = 2 * torch.sin(math.pi / 2 * x[:, 0]) * torch.cos(math.pi / 2 * x[:, 1])
    return h


def func_w(x):  # returns 1 for positions in the domain and 0 otherwise
    lens = x.shape[0]
    w_bool = torch.gt(1 - torch.abs(x[:, 0]), torch.zeros(lens).to(device)) & torch.gt(torch.abs(x[:, 0]), torch.zeros(lens).to(device))
    w_val = torch.where(w_bool, 1 - torch.abs(x[:, 0]) + torch.abs(x[:, 0]), torch.zeros(lens).to(device))
    return (w_val.view(-1, 1))


"""# Domain"""

T0 = 0  # if this is ignored it is always set as T0=0
T = 1

# Set up for a square
up = 1.0
down = -1.0
dim = 2
domain_sample_size = 8000  # 25000
t_mesh_size = 11
boundary_sample_size = 40  # 250

# defining the training domain
x0_domain = torch.Tensor(domain_sample_size, dim).uniform_(down, up)
x0_domain.requires_grad_()

x_domain_train = x0_domain.unsqueeze(2).repeat(1, 1, t_mesh_size)

t = torch.linspace(T0, T, t_mesh_size).unsqueeze(1).unsqueeze(2).view(1, 1, t_mesh_size).repeat(domain_sample_size, 1, 1)
xt_domain_train = torch.cat((x_domain_train, t), dim=1)

xt_domain_train = xt_domain_train.to(device)

# defining the training boundary
x0_boundary_side = torch.Tensor(boundary_sample_size, dim - 1).uniform_(down, up)
x0_boundary_side.requires_grad_()

x0_boundary_left = torch.cat((torch.ones(x0_boundary_side.size()) * down, x0_boundary_side), 1)
x0_boundary_right = torch.cat((torch.ones(x0_boundary_side.size()) * up, x0_boundary_side), 1)
x0_boundary_down = torch.cat((x0_boundary_side, torch.ones(x0_boundary_side.size()) * down), 1)
x0_boundary_up = torch.cat((x0_boundary_side, torch.ones(x0_boundary_side.size()) * up), 1)

x0_boundary = torch.cat((x0_boundary_left, x0_boundary_right, x0_boundary_down, x0_boundary_up), 0)

x_boundary_train = x0_boundary.unsqueeze(2).repeat(1, 1, t_mesh_size)
xt_boundary_train = torch.cat((x_boundary_train, t[:4*boundary_sample_size, :, :]), dim=1)
xt_boundary_train = xt_boundary_train.to(device)


# Validation data Sets
val_domain_size = int(domain_sample_size * 0.3)
val_boundary_size = int(boundary_sample_size * 0.3)

x0_domain_val = torch.Tensor(val_domain_size, dim).uniform_(down, up)
x0_domain_val.requires_grad_()

x_domain_val = x0_domain_val.unsqueeze(2).repeat(1, 1, t_mesh_size)
xt_domain_val = torch.cat((x_domain_val, t[:val_domain_size, :, :]), dim=1).to(device)

# defining the validation boundary
x0_boundary_side = torch.Tensor(val_boundary_size, dim - 1).uniform_(down, up)
x0_boundary_side.requires_grad_()

x0_boundary_left = torch.cat((torch.ones(x0_boundary_side.size()) * down, x0_boundary_side), 1)
x0_boundary_right = torch.cat((torch.ones(x0_boundary_side.size()) * up, x0_boundary_side), 1)
x0_boundary_down = torch.cat((x0_boundary_side, torch.ones(x0_boundary_side.size()) * down), 1)
x0_boundary_up = torch.cat((x0_boundary_side, torch.ones(x0_boundary_side.size()) * up), 1)

x0_boundary = torch.cat((x0_boundary_left, x0_boundary_right, x0_boundary_down, x0_boundary_up), 0)

x_boundary_val = x0_boundary.unsqueeze(2).repeat(1, 1, t_mesh_size)
xt_boundary_val = torch.cat((x_boundary_val, t[:4*val_boundary_size, :, :]), dim=1).to(device)

xv = xt_domain_train[:, 0, :].clone().detach()
yv = xt_domain_train[:, 1, :].clone().detach()
tv = xt_domain_train[:, 2, :].clone().detach()
xv = xv.requires_grad_(True).to(device)
yv = yv.requires_grad_(True).to(device)
tv = tv.requires_grad_(True).to(device)


xu = xv.clone().detach()
yu = yv.clone().detach()
tu = tv.clone().detach()
xu.requires_grad_(True).to(device)
yu.requires_grad_(True).to(device)
tu.requires_grad_(True).to(device)

x_val = xt_domain_val[:, 0, :].clone().detach().requires_grad_(True).to(device)
y_val = xt_domain_val[:, 1, :].clone().detach().requires_grad_(True).to(device)
t_val = xt_domain_val[:, 2, :].clone().detach().requires_grad_(True).to(device)

"""# Defining the Model"""


class generator(torch.nn.Module):  # this makes the u function
    def __init__(self, config):
        super().__init__()
        self.num_layers = config['u_layers']
        self.hidden_dim = config['u_hidden_dim']
        self.input = torch.nn.Linear(dim+1, self.hidden_dim)
        self.hidden = torch.nn.Linear(self.hidden_dim, self.hidden_dim)
        self.output = torch.nn.Linear(self.hidden_dim, 1)
        self.net = torch.nn.Sequential(*[
            self.input,
            *[torch.nn.ReLU(), self.hidden] * self.num_layers,
            torch.nn.Tanh(),
            self.output

        ])

    def forward(self, x0, y0, t):
        #t = t.unsqueeze(1).unsqueeze(2).view(1, 1, t_mesh_size).repeat(1, 2, 1)
        #x_ = torch.cat((x0.unsqueeze(2).view(-1, 1, t_mesh_size), y0.unsqueeze(2).view(-1, 1, t_mesh_size)), dim=1)
        inp = torch.cat((x0.unsqueeze(2), y0.unsqueeze(2), t.unsqueeze(2)), dim=2)
        x = self.net(inp)
        return x

    def backward(self, retain_graph=True):
        self.loss.backward(retain_graph=retain_graph)
        return (self.loss)


class discriminator(torch.nn.Module):  # this makes the v function
    def __init__(self, config):
        super().__init__()
        self.num_layers = config['v_layers']
        self.hidden_dim = config['v_hidden_dim']
        self.input = torch.nn.Linear(dim+1, self.hidden_dim)
        self.hidden = torch.nn.Linear(self.hidden_dim, self.hidden_dim)
        self.output = torch.nn.Linear(self.hidden_dim, 1)
        self.net = torch.nn.Sequential(*[
            self.input,
            *[torch.nn.ReLU(), self.hidden] * self.num_layers,
            torch.nn.Tanh(),
            self.output

        ])

    def forward(self, x0, y0, t):
        # t = t.unsqueeze(1).unsqueeze(2).view(1, 1, t_mesh_size).repeat(1, 2, 1)
        # x_ = torch.cat((x0.unsqueeze(2).view(-1, 1, t_mesh_size), y0.unsqueeze(2).view(-1, 1, t_mesh_size)), dim=1)
        inp = torch.cat((x0.unsqueeze(2), y0.unsqueeze(2), t.unsqueeze(2)), dim=2)
        x = self.net(inp)
        return x

    def backward(self, retain_graph=True):
        self.loss.backward(retain_graph=retain_graph)
        return (self.loss)
'''
#Search space based on the WAN Paper
config = {
    'alpha': tune.loguniform(1e3, 1e7),
    'u_layers': 7,
    'u_hidden_dim': 20,
    'v_layers': 7,
    'v_hidden_dim': 50,
    'n1': 2,  # tune.choice([2, 4, 6, 8, 10, 14, 20]),
    'n2': 1,  # tune.sample_from(lambda spec: int(spec.config.n1/2)),
    'u_rate': tune.loguniform(0.0001, 0.1),
    'v_rate': tune.loguniform(0.0001, 0.1),
    'num_workers': 2,
    'num_cpu_per_worker': 1
}
'''

#Result of Hyperparameter search
config = {
    'u_layers': 7,
    'u_hidden_dim': 20,
    'v_layers': 7,
    'v_hidden_dim': 50,
    'n1': 10,
    'n2': 5,
    'alpha': 30.296,
    'u_rate': 0.0032223,  # 0.00015
    'v_rate': 0.0064395  # 0.00015
}

'''
#Original WAN paper
config = {
    'u_layers': 7,
    'u_hidden_dim': 20,
    'v_layers': 7,
    'v_hidden_dim': 50,
    'n1': 10,  # 2,
    'n2': 5,  # 6, 1
    'alpha': 1e5,
    'u_rate': 0.015,  # 0.00015
    'v_rate': 0.04   # 0.00015
}

#One hyperparameter result
config = {
    'u_layers': 5,
    'u_hidden_dim': 20,
    'v_layers': 6,
    'v_hidden_dim': 16,
    'n1': 10,
    'n2': 6,
    'u_rate': 0.0025703,
    'v_rate': 0.00593051}

#Search Space for the above
config = {
    'u_layers': tune.choice([4, 5, 6, 7, 8]),
    'u_hidden_dim': tune.choice([20, 21, 23]),
    'v_layers': tune.choice([5, 6, 7, 8]),
    'v_hidden_dim': tune.choice([13, 15, 16]),
    'n1': 10,
    'n2': 5,
    'u_rate': tune.loguniform(1e-3, 1e-2),
    'v_rate': tune.loguniform(1e-3, 1e-2)
}
'''

"""# Loss Function"""

# TODO: ensure loss is normalised
def I(y_output_u, y_output_v, xt, xv, yv, tv, xu, yu, tu):
    shape = [y_output_u.shape[0], t_mesh_size, dim]
    shape[-1] = shape[-1] - 1
    y_output_u.retain_grad()
    y_output_v.retain_grad()
    phi = y_output_v * func_w(xt[:, :, 0]).unsqueeze(2).repeat(1, t_mesh_size, 1)
    y_output_u.backward(torch.ones(shape).to(device), retain_graph=True)
    du_x = xu.grad
    du_y = yu.grad
    phi.backward(torch.ones(shape).to(device), retain_graph=True)
    dphi_x = xv.grad
    dphi_y = yv.grad
    dphi_t = tv.grad.unsqueeze(2)
    s1 = y_output_u[:, -1, :] * phi[:, -1, :] - func_h(xt[:, 0, :]).unsqueeze(1) * phi[:, 0, :]
    s2 = (y_output_u * dphi_t)/t_mesh_size  # for t does this make sense?
    Lap = du_x * dphi_x + du_y * dphi_y
    s3 = (T-T0)*(Lap.unsqueeze(2) + y_output_u * y_output_u * phi - func_f(xt).unsqueeze(2) * phi)/t_mesh_size
    I = torch.sum(s1 - torch.sum(s2 - s3, 1), 0)
    xu.grad.data.zero_()
    yu.grad.data.zero_()
    xv.grad.data.zero_()
    yv.grad.data.zero_()
    tv.grad.data.zero_()
    return I

def L_init(y_output_u, config):
    #u_net = generator(config)
    return torch.mean((y_output_u[:, 0, 0] - func_h(xt_domain_train[:, :2, 0])) ** 2)


def L_bdry(u_net):
    return torch.mean((u_net(xt_boundary_train[:, 0, :], xt_boundary_train[:, 1, :], xt_boundary_train[:, 2, :]) -
                       func_g(xt_boundary_train).unsqueeze(2)) ** 2)


def L_int(y_output_u, y_output_v, xt=xt_domain_train, xv=xv, yv=yv, tv=tv, xu=xu, yu=yu, tu=tu):
    # x needs to be the set of points set plugged into net_u and net_v
    return torch.log((I(y_output_u, y_output_v, xt, xv, yv, tv, xu, yu, tu)) ** 2) - torch.log(torch.sum(y_output_v ** 2))


#gamma = 25    # 1e5*boundary_sample_size*4  # 25
#alpha = gamma


def L(y_output_u, y_output_v, u_net, alpha, gamma):
    return L_int(y_output_u, y_output_v) + gamma * L_init(y_output_u, config) + alpha * L_bdry(
        u_net)


def Loss_u(y_output_u, y_output_v, u_net, alpha, gamma):
    return L(y_output_u, y_output_v, u_net, alpha, gamma)


def Loss_v(y_output_u, y_output_v):
    return -L_int(y_output_u, y_output_v)

# TODO: identify the best n1, n2 (n2=n1/2) and best alpha and gamma

"""# Training"""

iteration = 2000

x_mesh = torch.linspace(0, 1, 50, requires_grad=True)
mesh1, mesh2 = torch.meshgrid(x_mesh, x_mesh)
mesh_1= torch.reshape(mesh1, [-1,1])
mesh_2= torch.reshape(mesh2, [-1,1])
t = torch.ones(2500, 1)
xt_test = torch.cat((mesh_1, mesh_2, t), dim = 1).unsqueeze(2)

def train(config, checkpoint_dir=None):
    n1 = config['n1']
    n2 = config['n2']

    # neural network models
    u_net = generator(config).to(device)
    v_net = discriminator(config).to(device)

    # optimizers for WAN
    optimizer_u = torch.optim.Adam(u_net.parameters(), lr=config['u_rate'])
    optimizer_v = torch.optim.Adam(v_net.parameters(), lr=config['v_rate'])

    #scheduler_u = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer_u, factor=0.5, patience=30)
    #scheduler_v = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer_v, factor=0.5, patience=30)

    prediction_u = u_net(xu, yu, tu)
    prediction_v = v_net(xv, yv, tv)

    Loss = 0

    for k in range(iteration):

        for i in range(n1):
            loss_u = Loss_u(prediction_u, prediction_v, u_net, config['alpha'], config['alpha'])
            optimizer_u.zero_grad()
            loss_u.backward(retain_graph=True)
            optimizer_u.step()
            #scheduler_u.step(loss_u)
            prediction_u = u_net(xu, yu, tu)

        for j in range(n2):
            loss_v = Loss_v(prediction_u, prediction_v)
            optimizer_v.zero_grad()
            loss_v.backward(retain_graph=True)
            optimizer_v.step()
            #scheduler_v.step(loss_v)
            prediction_v = v_net(xv, yv, tv)

        Loss += 0.1*loss_u

        if k % 10 == 0:
            #torch.save(u_net.state_dict(), "./net_u.pth")
            #torch.save(v_net.state_dict(), "./net_v.pth")
            print(k, loss_u.data, loss_v.data)
            # print('learning rate at %d epoch：%f' % (k, optimizer_u.param_groups[0]['lr']))
            # print('learning rate at %d epoch：%f' % (k, optimizer_v.param_groups[0]['lr']))
            #tune.report(Loss=Loss.item())
            Loss = 0
            error_test = torch.mean(
                torch.sqrt(torch.square((func_u_sol(xt_domain_train) - prediction_u.data.squeeze(2))))).data
            print("error test " + str(error_test))
            '''
            print(device)
            with tune.checkpoint_dir(k) as checkpoint_dir:
                path = os.path.join(checkpoint_dir, "checkpoint")
                torch.save((u_net.state_dict(), v_net.state_dict()), path)
            
            #PATHg = os.path.join('/Users/paulvalsecchi/Desktop/Project/Models alpha=25', 'generator_{}.pth'.format(k))
            #PATHd = os.path.join('/Users/paulvalsecchi/Desktop/Project/Models alpha=25', 'discriminator_{}.pth'.format(k))

            #torch.save(u_net.state_dict(), PATHg)
            #torch.save(v_net.state_dict(), PATHd)


        #error_test = torch.mean(torch.sqrt(torch.square((func_u_sol(xt_domain_train) - prediction_u.data.squeeze(2))))).data
        error_test = torch.mean(torch.sqrt(torch.square(func_u_sol(xt_domain_val)-u_net(x_val, y_val, t_val).data.squeeze(2))))
        #print(error_test)
        #tune.report(Loss=float(loss_u.detach().numpy()))


        #EarlyStopping(loss_u, u_net)
        '''

train(config)


'''
u_net = generator(config)
PATHg = os.path.join('/Users/paulvalsecchi/Desktop/Project/Models', 'generator_1000.pth')
u_net.load_state_dict(torch.load(PATHg))
u_net.eval()

'''
plt.plot(func_u_sol(xt_test).data.numpy())
plt.plot(u_net(xt_test[:, 0, 0].unsqueeze(1), xt_test[:, 1, 0].unsqueeze(1), xt_test[:, 2, 0].unsqueeze(1)).squeeze(2).data.numpy())
plt.show()

'''
# Hyperparameter Optimization

# Best trial config: {'u_layers': 5, 'u_hidden_dim': 20, 'v_layers': 5, 'v_hidden_dim': 5, 'n1': 10, 'n2': 5, 'u_rate': 0.0002133670132946601, 'v_rate': 0.0006349874886164373}
# Best trial config: {'u_layers': 2, 'u_hidden_dim': 20, 'v_layers': 4, 'v_hidden_dim': 20, 'n1': 10, 'n2': 7, 'u_rate': 0.0006756995220370141, 'v_rate': 0.0002804879090959305}
# Best trial config: {'u_layers': 4, 'u_hidden_dim': 23, 'v_layers': 4, 'v_hidden_dim': 15, 'n1': 10, 'n2': 6, 'u_rate': 0.003370553415547106, 'v_rate': 0.009087847200586583}
# Best trial config: {'u_layers': 7, 'u_hidden_dim': 30, 'v_layers': 7, 'v_hidden_dim': 50, 'n1': 10, 'n2': 6, 'u_rate': 0.07933644599535282, 'v_rate': 0.03159263256623099}


analysis = tune.run(
    train,
    num_samples=200,
    scheduler=ASHAScheduler(metric="Loss", mode="min", grace_period=10, max_t=200),
    config=config,
    verbose=2,
    resources_per_trial = {"cpu":2} #,"gpu":1}
)

best_trial = analysis.get_best_trial(metric="Loss", mode="min")
print("Best trial config: {}".format(best_trial.config))

best_trained_generator = generator(best_trial.config)
best_trained_discriminator = discriminator(best_trial.config)

best_checkpoint_dir = best_trial.checkpoint.value
generator_state, discriminator_state = torch.load(os.path.join(
    best_checkpoint_dir, "checkpoint"))



# Obtain a trial dataframe from all run trials of this `tune.run` call.
dfs = analysis.trial_dataframes


ax = None  # This plots everything on the same plot
for d in dfs.values():
    ax = d.mean_error.plot(ax=ax, legend=False)
ax.set_xlabel("Epochs")
ax.set_ylabel("Mean Error")


x_mesh = torch.linspace(down, up, 500, requires_grad=True)
mesh1, mesh2 = torch.meshgrid(x_mesh, x_mesh)
mesh_1 = torch.reshape(mesh1, [-1, 1]).repeat(1, 11).unsqueeze(2).view(-1, 1, 11)
mesh_2 = torch.reshape(mesh2, [-1, 1]).repeat(1, 11).unsqueeze(2).view(-1, 1, 11)
t = torch.linspace(0, 1, 11).unsqueeze(1).view(1, -1).repeat(250000, 1).unsqueeze(2).view(-1, 1, 11)
xt_comparison = torch.cat((mesh_1, mesh_2, t), dim=1)

u_net = generator(config)
'''

error = torch.sqrt(torch.square(func_u_sol(xt_comparison)-u_net(xt_comparison[:, 0, :], xt_comparison[:, 1, :], xt_comparison[:, 2, :]).squeeze(2)))
# mean_error = torch.mean(error, dim=1)
# mean_error = mean_error.view(500, 500)
l_error = error[:, -1].view(500,500)

plt.figure(figsize=(10, 6))

cset=plt.contourf(mesh1.data.numpy(),mesh2.data.numpy(), l_error.data.numpy(), 500, cmap='winter')

plt.colorbar(cset)
plt.show()


cuda
True
0 tensor([133.1307], device='cuda:0') tensor([-7.0305], device='cuda:0')
error test tensor(1.3880, device='cuda:0')
10 tensor([16.1506], device='cuda:0') tensor([-7.8067], device='cuda:0')
error test tensor(0.6644, device='cuda:0')
20 tensor([10.1781], device='cuda:0') tensor([-8.7759], device='cuda:0')
error test tensor(0.5381, device='cuda:0')
30 tensor([9.5432], device='cuda:0') tensor([-8.1748], device='cuda:0')
error test tensor(0.5721, device='cuda:0')
40 tensor([9.6108], device='cuda:0') tensor([-8.7363], device='cuda:0')
error test tensor(0.5929, device='cuda:0')
50 tensor([9.5738], device='cuda:0') tensor([-7.8691], device='cuda:0')
error test tensor(0.6181, device='cuda:0')
60 tensor([9.3341], device='cuda:0') tensor([-8.9492], device='cuda:0')
error test tensor(0.6366, device='cuda:0')
70 tensor([9.3182], device='cuda:0') tensor([-8.8181], device='cuda:0')
error test tensor(0.6603, device='cuda:0')
80 tensor([9.4617], device='cuda:0') tensor([-8.8898], device='cuda